In [102]:
# Import the two datasets and drop subjects whose individual estimates do not converge
from src.config import SRC, BLD
import pandas as pd
import numpy as np
from scipy.stats import norm
from estimagic import estimate_ml

data = pd.read_stata(SRC / "original_data" / "decisions_data.dta")    # full sample
ind_keep = pd.read_csv(SRC / "original_data" / "ind_to_keep.csv")   # import csv with ID of subjects to keep 
data = data[data.wid.isin(ind_keep.wid_col1)] # this is the primary sample for the aggregate estimates (72 individuals)
data = data[data.bonusoffered != 1]  # remove observations where a bonus was offered
data["pb"] = data["workdone1"] / 10
    # workdone1 can either be 10 or 0: dividing the variable by 10 creates the dummy
data["ind_effort10"] = (data["effort"] == 10).astype(int)  # ind_effort10 dummy
data["ind_effort110"] = (data["effort"] == 110).astype(int)  # ind_effort110 dummy
data.index = np.arange(len(data.wid))


def negloglike(params, data):
    
    
    # We use np.array to allow for element-wise operations # make that piece of code better into one function
    
    
    predchoice = ((params.loc["phi", "value"]*(params.loc["delta", "value"]**data["netdistance"])*(params.loc["beta", "value"]**data["today"])*(params.loc["betahat", "value"]**data["prediction"])*data["wage"])**(1/(params.loc["gamma", "value"]-1)))-data["pb"]*params.loc["alpha", "value"]
    
    prob = (1-data["ind_effort10"]-data["ind_effort110"])*norm.pdf(data["effort"], predchoice, params.loc["sigma", "value"])+data["ind_effort10"]*(1 - norm.cdf((predchoice-data["effort"])/params.loc["sigma", "value"]))+data["ind_effort110"]*norm.cdf((predchoice-data["effort"])/params.loc["sigma", "value"])
        
    index_p0 = [i for i in range(0,len(prob)) if prob[i]==0] # vector containing the indexes when prob=0
    index_p1 = [i for i in range(0,len(prob)) if prob[i]==1] # vector containing the indexes when prob=1
    
    # use a for loop to change the values   # change this to list comprehension
    
    for i in index_p0:
        prob[i] = 1E-4
        
    for i in index_p1:
        prob[i] = 1 - 1E-4
    
    contr = np.log(prob)
    
    return { "contributions": contr , "value": np.sum(contr)}


def start_params():
    """
    Define initial guesses Consistent with the ones used by Augenblick & Rabin in original paper
    and Pozzi & unnari in their replication  

    """
    parm =[0.8,1,1,2,500,7,40]
    init_parm = pd.DataFrame(parm, columns = ["value"], index = ["beta", "betahat", "delta", "gamma", "phi", "alpha", "sigma"])
    return init_parm

def load_args():
    netdistance = np.array(data["netdistance"])
    wage = np.array(data["wage"])
    today = np.array(data["today"])
    prediction = np.array(data["prediction"])
    pb = np.array(data["pb"])
    effort = np.array(data["effort"])
    ind_effort10 = np.array(data["ind_effort10"])
    ind_effort110 = np.array(data["ind_effort110"])
    return pd.DataFrame({"netdistance": netdistance, "wage":wage, "today": today , "prediction": prediction, "pb" : pb ,"effort": effort, "ind_effort10": ind_effort10,"ind_effort110": ind_effort110})

params = start_params()
data = load_args()



res = estimate_ml(
    loglike = negloglike,
    params = start_params(),
    optimize_options = {"algorithm": "scipy_neldermead"},
    loglike_kwargs={"data": data}, 
)

res["summary_jacobian"].round(3)


C:\Anaconda\anaconda3\envs\replication_ar2018\lib\site-packages\estimagic\utilities.py:193: UserWarning: Standard matrix inversion failed due to LinAlgError described below. A pseudo inverse was calculated instead. Taking the inverse of the information matrix failed. Only ever use this covariance matrix or standard errors based on it for diagnostic purposes, not for drawing conclusions.
  warnings.warn(header + msg)
C:\Anaconda\anaconda3\envs\replication_ar2018\lib\site-packages\estimagic\inference\shared.py:116: RuntimeWarning: invalid value encountered in sqrt
  free["standard_error"] = np.sqrt(np.diag(free_cov))


,value,standard_error,p_value,ci_lower,ci_upper,stars
beta,0.835,0.005,0.0,0.826,0.844,***
betahat,0.999,0.006,0.0,0.988,1.010,***
delta,1.003,NaN,NaN,NaN,NaN,NaN
gamma,2.145,0.002,0.0,2.141,2.150,***
phi,723.974,0.000,0.0,723.974,723.974,***
alpha,7.307,0.918,0.0,5.507,9.107,***
sigma,42.625,0.444,0.0,41.755,43.494,***
